In [1]:
import matplotlib.pyplot as plt

import pyBeamSim
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

import torch.optim as optim
from model.old_old_model import Model, Model2, MLP

from torchsummary import summary

from deap import base, creator, tools, algorithms


In [2]:
df = pd.read_csv('../data/mlp_10000.csv', index_col=0)
df

,drift1_len,quad1_len,quad1_gra,drift2_len,quad2_len,quad2_gra,x_sig4,y_sig4
0,0.402310,0.450841,-16.332723,0.170125,0.213844,7.026671,0.011611,0.012997
1,0.237463,0.454075,14.937412,0.273825,0.334553,5.617945,0.004942,0.007303
2,0.414345,0.177982,19.552067,0.388376,0.442580,2.004069,0.004739,0.004655
3,0.246283,0.124380,0.320149,0.130190,0.210073,13.761691,0.003692,0.013368
4,0.256067,0.194462,6.097415,0.473669,0.432795,7.627331,0.001405,0.006946
...,...,...,...,...,...,...,...,...
9995,0.247073,0.286787,2.478341,0.375353,0.320700,-13.608154,0.011704,0.003130
9996,0.407613,0.432325,4.456587,0.302943,0.444821,4.182542,0.003308,0.008749
9997,0.148527,0.189331,10.690400,0.205502,0.385139,9.790066,0.001836,0.007299
9998,0.307523,0.318801,2.121998,0.348172,0.262224,-10.475280,0.014424,0.003703


In [3]:
ref_energy = 5
num_particle = 102400
simulator = pyBeamSim.BeamSimulator()
simulator.init_beam(num_particle, 938.272046, 1.0, 0.0)
# key step
simulator.init_Beamline()
simulator.set_beamTwiss(0, 0.003, 0.0001, 0, 0.003, 0.0001, 0, 8, 3.1415926e-11, 0, ref_energy, 500, 1)
simulator.UpdateBeamParameters()

def simulate_by_seq(simulator, value_list):
    simulator.init_Beamline()
    simulator.add_Drift('drift1', value_list[0], Aperture=0.05)
    simulator.add_Quad('quad1', value_list[1], Aperture=0.012, FieldGradient=value_list[2])
    simulator.add_Drift('drift2', value_list[3], Aperture=0.05)
    simulator.add_Quad('quad2', value_list[4], Aperture=0.028, FieldGradient=value_list[5])
    simulator.simulate_all()
    simulator.UpdateBeamParameters()
    x_sig = simulator.getSigX()
    y_sig = simulator.getSigY()
    return x_sig, y_sig

In [4]:
f= [0.237463,0.454075,14.937412,0.273825,0.334553,5.617945]
simulate_by_seq(f)

(0.0005616418695696335, 0.0008292177069427745)

In [5]:
a,b = simulate_by_seq(front[0][0])
print(a,b,'\n')
print(front[0][0].fitness.values, '\n')
c, d = simulate_by_seq(front[0][5])
print(c, d)
print(front[0][5].fitness.values, '\n')

NameError: name 'front' is not defined

In [9]:
"""" variable dataframe created"""
vary = ['drift1_len', 'quad1_len', 'quad1_gra', 'drift2_len', 'quad2_len', 'quad2_gra']
low = [0.1, 0.1, -20.0, 0.1, 0.1, -20.0]
high = [0.5, 0.5, 20.0, 0.5, 0.5, 20.0]
units = ['m', 'm', 'T/m', 'm', 'm', 'T/m']
variable = pd.DataFrame(index=vary)
variable['low'] = low
variable['high'] = high
variable['units'] = units
variable

,low,high,units
drift1_len,0.1,0.5,m
quad1_len,0.1,0.5,m
quad1_gra,-20.0,20.0,T/m
drift2_len,0.1,0.5,m
quad2_len,0.1,0.5,m
quad2_gra,-20.0,20.0,T/m


In [10]:
X = df.iloc[:, :-2]
y = df.iloc[:, -2:]
X = torch.Tensor(X.values)
y = torch.Tensor(y.values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

In [11]:
batch_size = 32

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

In [12]:
# model:MLP(Depth: 4, Width:6-32-64-32-16-2, learning_rate: 0.001 )
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
criterion = nn.L1Loss()

model = nn.Sequential(nn.Linear(6, 32),
                    nn.ReLU(),
                    nn.Dropout(0.1),
                    nn.Linear(32, 64),
                    nn.ReLU(),
                    nn.Dropout(0.1),
                    nn.Linear(64, 32),
                    nn.ReLU(),
                    nn.Dropout(0.1),
                    nn.Linear(32, 16),
                    nn.ReLU(),
                    nn.Dropout(0.1),
                    nn.Linear(16, 2),
                    )

model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0013, betas=(0.9, 0.999))

In [13]:
def val_acc(ml, val_loader, device):
    ml.eval()
    total_acc = 0
    with torch.no_grad():
        for X_test, y_test in val_loader:
            X_test = X_test.to(device)
            y_test = y_test.to(device)
            y_pred = ml(X_test)
            # test_criterion = nn.MSELoss(reduction='sum')
            test_criterion = nn.L1Loss()
            acc = test_criterion(y_pred, y_test)
            total_acc += acc
    return total_acc

In [14]:
# train , if not ,just torch.load

num_epochs=50

losses = []
test_accuracies = []

test_acc = val_acc(model, val_loader=test_loader, device=device)
test_accuracies.append(test_acc.cpu().numpy())

for epoch in range(num_epochs):
    if epoch == 0:
        print(f'the training is using {device}')
    model.train()
    total_loss = 0
    for X, y in train_loader:
        X = X.to(device)
        y = y.to(device) # two-dim tensor
        y_pred = model(X)
        loss = criterion(y_pred, y)
        total_loss += loss
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    test_acc = val_acc(model, val_loader=test_loader, device=device)
    test_accuracies.append(test_acc.cpu().numpy())
    losses.append(total_loss.cpu().detach().numpy()) # to cpu to plot
    print(f'epoch{epoch+1}: loss: {total_loss:8f}')

torch.save(model.state_dict(), '../model_save/deap_mlp.pth')

the training is using cuda
epoch1: loss: 4.341819
epoch2: loss: 0.982373
epoch3: loss: 0.738598
epoch4: loss: 0.694039
epoch5: loss: 0.686818
epoch6: loss: 0.674485
epoch7: loss: 0.665541
epoch8: loss: 0.663569
epoch9: loss: 0.650530
epoch10: loss: 0.644949
epoch11: loss: 0.645172
epoch12: loss: 0.637503
epoch13: loss: 0.638533
epoch14: loss: 0.632399
epoch15: loss: 0.630358
epoch16: loss: 0.629511
epoch17: loss: 0.620229
epoch18: loss: 0.608286
epoch19: loss: 0.602227
epoch20: loss: 0.574361
epoch21: loss: 0.560060
epoch22: loss: 0.539703
epoch23: loss: 0.538190
epoch24: loss: 0.531964
epoch25: loss: 0.525708
epoch26: loss: 0.517876
epoch27: loss: 0.503357
epoch28: loss: 0.506303
epoch29: loss: 0.501459
epoch30: loss: 0.493267
epoch31: loss: 0.486050
epoch32: loss: 0.488408
epoch33: loss: 0.474725
epoch34: loss: 0.469560
epoch35: loss: 0.469140
epoch36: loss: 0.456870
epoch37: loss: 0.455961
epoch38: loss: 0.451620
epoch39: loss: 0.451670
epoch40: loss: 0.450997
epoch41: loss: 0.44370

In [ ]:
model.load_state_dict(torch.load('../model_save/deap_mlp.pth'))
model.to('cpu')

In [ ]:
# test
x = X[0, :].to('cpu').detach()
print(model(x))
print(y[2, :])  

In [ ]:
model.eval()
model(torch.Tensor([0, 0, 0, 0, 0, 0])).detach().numpy()[0]

### NSGA-Ⅱ

In [ ]:
# for speed, occur in cpu or gpu

model.to('cpu')
model.eval()


def create_individual():
    return [np.random.uniform(row['low'], row['high']) for _, row in variable.iterrows()]

def evaluate(individual):
    output = model(torch.tensor(individual)).detach().numpy()
    return output[0], output[1]

def genInd(low, high):
    return [random.uniform(low_i, high_i) for low_i, high_i in zip(low, high)]
# def create_individual():
#     return [np.random.uniform(row['low'], row['high']) for _, row in variable.iterrows()]

In [ ]:
# define question
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))   # two min
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
low = [0.1, 0.1, -20.0, 0.1, 0.1, -20.0]
high = [0.5, 0.5, 20.0, 0.5, 0.5, 20.0]
n_genes = len(low)  # 6

# initial individual and population
toolbox.register('genInd',genInd,low,high)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.genInd)   # list
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=low, up=high, eta=0.5)
toolbox.register("mutate", tools.mutPolynomialBounded, low=low, up=high, eta=20.0, indpb=0.2)
toolbox.register("select", tools.selNSGA2)


In [ ]:
N_POP = 200#种群内个体数量
ngen = 100#迭代步数，参数过小，在收敛之前就结束搜索
cxpb = 0.8#交叉概率，参数过小，族群不能有效更新
mutpb = 0.2#突变概率，参数过小，容易陷入局部最优
hof = tools.ParetoFront()

population = toolbox.population(n=N_POP)
for ind in population:
    ind.fitness.values = toolbox.evaluate(ind)
for gen in range(1, ngen + 1):

    # 选择下一代的父母
    offspring = toolbox.select(population, len(population))

    # 克隆选择出来的个体
    offspring = list(map(toolbox.clone, offspring))

    # 应用交叉和变异
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < cxpb:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < mutpb:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # 重新评估所有失效的个体
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    for ind in invalid_ind:
        ind.fitness.values = toolbox.evaluate(ind)

    # 将当前种群和后代种群合并
    population[:] = toolbox.select(population + offspring, len(population))

    # 更新 Pareto 前沿
    hof.update(population)

    # 输出当前代的最优解
    front = tools.sortNondominated(population, len(population), first_front_only=True) # one tuple  with a list if True
    print(f"num in Generation {gen}: {len(front[0])}")
    # for ind in front[0]:
    #     print(ind.fitness.values, ind)
    
# fronts = tools.emo.sortNondominated(pop,k=N_POP,first_front_only=False)#快速非支配排序，得到不同前沿的pareto层集合fronts
# #print(fronts)#打印一个pareto层集合fronts检查，每层前沿都是一个列表


### get figure for pareto front

In [ ]:
dots = [] # for MLP
# true for simulator
true_x_sig = []
true_y_sig = []
for inv in front[0]:
    dots.append(inv.fitness.values)
    x_sig, y_sig = simulate_by_seq(inv)
    true_x_sig.append(x_sig)
    true_y_sig.append(y_sig)
all_x = [point[0] for point in dots]
all_y = [point[1] for point in dots]


In [ ]:
plt.figure(figsize=(5, 5))
plt.title('pareto front')
plt.xlabel('x_sig')
plt.ylabel("y_sig")

plt.scatter(all_x, all_y)
plt.scatter(true_x_sig, true_y_sig, color='yellow', marker='x')

### test for failure

In [ ]:
# for speed, occur in cpu or gpu

model.to('cpu')
model.eval()

def model_predict(params):
    with torch.no_grad():
        return model(torch.tensor(params, dtype=torch.float32)).numpy()

# define question
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

# individual
def create_individual():
    return [np.random.uniform(row['low'], row['high']) for _, row in variable.iterrows()]

# fitness function
def evaluate(individual):
    output = model_predict(individual)
    return output[0], output[1]

# initial toolbox
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxSimulatedBinary, eta=0.5)   # 交叉
# toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
# toolbox.register("mutate", tools.mutPolynomialBounded, eta=20.0, low=[row['low'] for _, row in variable.iterrows()],
#                  up=[row['high'] for _, row in variable.iterrows()], indpb=0.2)
toolbox.register("mutate", tools.mutPolynomialBounded, eta=20.0, low=low, up=high, indpb=0.50)
toolbox.register("select", tools.selNSGA2)

# initial the population
population = toolbox.population(n=100)

NGEN = 200#迭代步数，参数过小，在收敛之前就结束搜索
CXPB = 0.8#交叉概率，参数过小，族群不能有效更新
MUTPB = 0.2#突变概率，参数过小，容易陷入局部最优

# epoch by NSGA
for gen in range(50): # 50 generations
     fitnesses = list(map(toolbox.evaluate, population))
     for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit
    # Selection, crossover, and mutation
     offspring = toolbox.select(population, len(population))
     offspring = list(map(toolbox.clone, offspring))
     
     for child1, child2 in zip(offspring[::2], offspring[1::2]):
         if np.random.rand() < 0.5:  # 交叉概率
             toolbox.mate(child1, child2)    #inplace 
             del child1.fitness.values  # 使适应度无效
             del child1.fitness.values
             
     for mutant in offspring:
         if np.random.rand() < 0.2:  # 变异概率
             toolbox.mutate(mutant)
             del mutant.fitness.values  # 使适应度无效

     # 更新种群
     population[:] = offspring
     
fits = [ind.fitness.values for ind in population]

In [ ]:
population = toolbox.population(n=10)
fitnesses = list(map(toolbox.evaluate, population))
fitnesses

In [ ]:
population = toolbox.population(n=100)
fitnesses = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit
    # Selection, crossover, and mutation
offspring = toolbox.select(population, len(population))
offspring = list(map(toolbox.clone, offspring))
for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if np.random.rand() < 0.5:  # 交叉概率
        toolbox.mate(child1, child2)
        del child1.fitness.values  # 使适应度无效
for mutant in offspring:
    if np.random.rand() < 0.2:  # 变异概率
        toolbox.mutate(mutant)
        del mutant.fitness.values  # 使适应度无效

In [ ]:
offspring

In [ ]:
create_individual()

In [ ]:
create_individual()